<a href="https://colab.research.google.com/github/fjadidi2001/Insurance/blob/main/Risk_Category_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.preprocessing import LabelEncoder

In [3]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.preprocessing import LabelEncoder

# Load your dataset
data = pd.read_csv('telematics_syn.csv')



# Step 1: Inspect the Dataset


In [4]:
# Check the first few rows
print(data.head())

# Check data types and missing values
print(data.info())

# Summary statistics
print(data.describe())

   Duration  Insured.age Insured.sex  Car.age  Marital  Car.use  Credit.score  \
0       366           45        Male       -1  Married  Commute         609.0   
1       182           44      Female        3  Married  Commute         575.0   
2       184           48      Female        6  Married  Commute         847.0   
3       183           71        Male        6  Married  Private         842.0   
4       183           84        Male       10  Married  Private         856.0   

  Region  Annual.miles.drive  Years.noclaims  ...  Left.turn.intensity10  \
0  Urban             6213.71              25  ...                    1.0   
1  Urban            12427.42              20  ...                   58.0   
2  Urban            12427.42              14  ...                    0.0   
3  Urban             6213.71              43  ...                    0.0   
4  Urban             6213.71              65  ...                    2.0   

   Left.turn.intensity11  Left.turn.intensity12  Right.t

# Step 2: Handle Missing Values


In [8]:
# Drop the single row with missing values
data.dropna(inplace=True)

## Step 2.1. Fix Invalid Car.age


In [9]:
# Replace negative values with 0 (assuming data entry error)
data['Car.age'] = data['Car.age'].clip(lower=0)

# 3. Encode Categorical Variables


In [10]:
# One-hot encode categorical features
cat_cols = ['Insured.sex', 'Marital', 'Car.use', 'Region']
data = pd.get_dummies(data, columns=cat_cols, drop_first=True)

# 4. Separate Features & Target
